In [1]:
'''
2018-11-20
following along with <https://github.com/skyfielders/astronomy-notebooks/blob/master/Solvers/Earth-Satellite-Passes.ipynb>
'''

import math

from skyfield.api import Loader, Topos, earth, JulianDate
import matplotlib.pyplot as plt
from scipy import optimize
import maidenhead as mh
import numpy as np

load = Loader('data/skyfield')

ts = load.timescale()

tau = 2.0 * np.pi

'''CONFIGURE ME!!!!'''
tle = load.tle('../tle/active.txt')
src_mh = 'EM15'
satellite = 'FOX-1B'
#satellite = 'AO-92'
window_start = (2018, 11, 24)
window_days = 5.0
min_altitude = 30.0
time_above = 15.0

# do every minute for 24 hours starting on 2018-11-19 0000Z UTC
window_minutes = 24 * 60 * window_days
time_range = ts.utc(*window_start, 0, range(int(window_minutes)))

home = Topos(*mh.toLoc(src_mh))
sat = tle[satellite]

diff = sat - home
print(diff)

def alt_f(t):
    return diff.at(t).altaz()[0].degrees

print(time_range)
print(time_range[0])
print(time_range[-1])
altitudes = alt_f(time_range)
plt.plot(time_range.tai, altitudes)
plt.title('{} elevation @ {}\n{} to {}'.format(
        satellite
        , src_mh
        , time_range[0].utc_iso()
        , time_range[-1].utc_iso()
    )
)

FileNotFoundError: [Errno 2] No such file or directory: 'data/skyfield/data/tle/active.txt'

In [ ]:
# sample six points per orbit to improve performance
orbit_period_per_minute = tau / sat.model.no
orbit_period_per_day = tau / sat.model.no / 24.0 / 60.0
print(orbit_period_per_day)
orbit_period = orbit_period_per_minute / window_minutes
revolutions_in_window = 1.0 / orbit_period # approximate number of peaks
sample_points = int(math.ceil(revolutions_in_window * 6.0))
#sample_step = int(math.floor(window_minutes / sample_points))
sample_step = (time_range[-1] - time_range[0]) / sample_points
print('sample_step', sample_step)
print(revolutions_in_window, 'revolutions in window')
print(orbit_period_per_minute, 'orbital period per minute')
print(orbit_period / 6.0, 'orbit period')
print(sample_points, 'sample points (revs*6)')
#sample_time_range = ts.utc(*window_start, 0, range(0, int(math.ceil(window_minutes)), sample_step))
sample_time_range = ts.tai_jd([time_range[0].tai + (_ * sample_step) for _ in range(sample_points)])
print(sample_time_range)

sample_altitudes = alt_f(sample_time_range)
print(sample_altitudes)

In [ ]:
left_diff = np.ediff1d(sample_altitudes, to_begin=0.0)
right_diff = np.ediff1d(sample_altitudes, to_end=0.0)
maxima = (left_diff > 0.0) & (right_diff < 0.0)
print(sum(maxima), 'sample peaks')
plt.plot(sample_time_range.tai, sample_altitudes)
plt.title('{} elevation @ {}\n{} minute sample\n{} to {}'.format(
        satellite
        , src_mh
        , sample_step
        , sample_time_range[0].utc_iso()
        , sample_time_range[-1].utc_iso()
    )
)
plt.plot(sample_time_range[maxima].tai, sample_altitudes[maxima], 'ro')
print(sample_time_range[maxima])

In [ ]:
def alt_f_minimization_wrapper(t):
    return -alt_f(ts.tai_jd(t))

def find_highest(t, step):
    return optimize.minimize_scalar(
        alt_f_minimization_wrapper
        , bracket=[t.tai + step, t.tai - step]
        , tol=(1.0 / 24.0 / 60.0 / 60.0) / t.tai
    ).x

print((window_minutes / sample_points) / (60 * 24))
#t_highest = ts.tai_jd([find_highest(ti, (window_minutes / sample_points) / (60 * 24)) for ti in sample_time_range[maxima]])
np.random.seed(99999)
t_highest = ts.tai_jd([find_highest(ti, sample_step) for ti in sample_time_range[maxima]])
print(t_highest)
plt.plot(sample_time_range.tai, sample_altitudes)
plt.title('{} elevation @ {}\n{} minute sample\n{} to {}'.format(
        satellite
        , src_mh
        , sample_step
        , sample_time_range[0].utc_iso()
        , sample_time_range[-1].utc_iso()
    )
)
plt.plot(sample_time_range[maxima].tai, sample_altitudes[maxima], 'ro')

In [ ]:
plt.title('{} elevation @ {}\n{} to {}'.format(
        satellite
        , src_mh
        , sample_time_range[0].utc_iso()
        , sample_time_range[-1].utc_iso()
    )
)
plt.plot(time_range.tai, altitudes)
plt.plot([_.tai for _ in t_highest], alt_f(t_highest), 'go')

In [ ]:
def rising_setting_wrapper(t):
    return -alt_f_minimization_wrapper(t)

def find_rising(t, step):
    return optimize.brentq(rising_setting_wrapper, t.tai - (2 * step), t.tai)

def find_setting(t, step):
    return optimize.brentq(rising_setting_wrapper, t.tai + (2 * step), t.tai)

print(t_highest)
passes = ts.tai_jd([_.tai for _ in t_highest if alt_f(_) > min_altitude])
print(passes)
rising = ts.tai_jd([find_rising(_, (window_minutes / sample_points) / (24 * 60)) for _ in passes])
setting = ts.tai_jd([find_setting(_, (window_minutes / sample_points) / (24 * 60)) for _ in passes])

for start, peak, stop in zip(rising, passes, setting):
    print(start.utc_iso(), peak.utc_iso(), stop.utc_iso())